In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from datasets import Dataset, Features, Value, ClassLabel

ModuleNotFoundError: No module named 'datasets'

In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd drive/My\ Drive/

UsageError: Line magic function `%` not found.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving prefab-breaker-411817-9e17e919dfef.json to prefab-breaker-411817-9e17e919dfef.json


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/prefab-breaker-411817-9e17e919dfef.json"

In [ ]:
!pip install google-cloud-translate

In [ ]:
! pip install datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00


In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df = df[['text', 'category']]
    return df

In [ ]:
def load_and_print_columns(file_path):
    df = pd.read_csv(file_path, nrows=4)
    print(f"Columns in {file_path}: {df.columns.tolist()}")
load_and_print_columns('homo_eng_3_train.csv')

Columns in homo_eng_3_train.csv: ['Unnamed: 0', 'text                ', 'category']


In [ ]:
smth = load_data('final_training_set_fully_balanced_shuffled.csv')

In [ ]:
print(smth)

                                                     text  \
0       #justiciaparasamuel😭💜😭💜😭💜😭💜😭💜💜🏳️‍🌈ILEGALIZACIÓ...   
1                    ఆల్‌ ఇండియా సర్వీసుకు హైకోర్టులు నో!   
2       ట్రాన్స్‌జెండర్‌లు అస్వాభావికులు, వారు మన సమాజ...   
3       उन्होंने कहा, &quot;अगर आप इसके प्रति आकर्षित ...   
4                            Who is here after insta ??😁😆   
...                                                   ...   
120496      ગેનો માર્ગ દુષ્કર્મ છે, તેમ કરવું દુષ્કર્મ છે   
120497  Phla insaan jo esi batein bhi bta rha h to bhi...   
120498                               రైల్వే శాఖకు విన్నపం   
120499  You tried to make a joke about a regular girl,...   
120500  ഒന്നുകിൽ പുരുഷനായി ജീവിക്കുകയോ സ്ത്രീയായി ജീവി...   

                      category  
0                   Homophobia  
1       Non-anti-LGBT+ content  
2                  Transphobia  
3                   Homophobia  
4       Non-anti-LGBT+ content  
...                        ...  
120496              Homophobia  
120

In [ ]:
train_tam_data = load_data('homo_tam_3_train.csv')

In [ ]:
train_eng_data = load_data('homo_eng_3_train.csv')
dev_eng_data = load_data('homo_eng_3_dev.csv')

train_tam_data = load_data('homo_tam_3_train.csv')
dev_tam_data = load_data('homo_tam_3_dev.csv')

train_mal_data = load_data('homo_mal_3_train.csv')
dev_mal_data = load_data('homo_mal_3_dev.csv')

train_hin_data = load_data('homo_hin_3_train.csv')
dev_hin_data = load_data('homo_hin_3_dev.csv')

train_telugu_data = load_data('homo_Telugu_train.csv')
dev_telugu_data = load_data('homo_Telugu_dev.csv')

train_kannada_data = load_data('homo_Kannada_train.csv')
dev_kannada_data = load_data('homo_Kannada_dev.csv')

train_gujarati_data = load_data('homo_Gujarati_train.csv')
dev_gujarati_data = load_data('homo_Gujarati_dev.csv')

train_marathi_data = load_data('Marathi_train.csv')
dev_marathi_data = load_data('Marathi_dev.csv')

train_spanish_data = load_data('Homo_Spanish_train.csv')
dev_spanish_data = load_data('Homo_Spanish_dev.csv')


In [ ]:
#train_spanish_data = load_data('Homo_Spanish_train.csv')
#dev_spanish_data = load_data('Homo_Spanish_dev.csv')

In [ ]:
print(train_eng_data[100:110])

                                                  text                category
100            Fully script story only                  Non-anti-LGBT+ content
101                   what an altitude                  Non-anti-LGBT+ content
102  Priya S   answer this  if Satan punishes bad g...  Non-anti-LGBT+ content
103  Dislike people of this video are non humanityD...              Homophobia
104  IN india how many of them folowing our culture...  Non-anti-LGBT+ content
105              Bro wating for part 2                  Non-anti-LGBT+ content
106  Love is Love And Everyone should be able to Ma...  Non-anti-LGBT+ content
107  Hes exposed that place front of public        ...  Non-anti-LGBT+ content
108  They are doing making public fully damage     ...              Homophobia
109  Best ever hearted one Live long Happy in this ...  Non-anti-LGBT+ content


In [ ]:
print(train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                 179
Transphobia                  7
Name: category, dtype: int64


In [ ]:
print(train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                 453
Transphobia                145
Name: category, dtype: int64


In [ ]:
print(train_mal_data['category'].value_counts())

Non-anti-LGBT+ content    2468
Homophobia                 476
Transphobia                170
Name: category, dtype: int64


In [ ]:
print(train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2423
Transphobia                 92
Homophobia                  45
Name: category, dtype: int64


In [ ]:
print(train_telugu_data['category'].value_counts())

Non-anti-LGBT+ content    3496
Homophobia                2907
Transphobia               2647
Name: category, dtype: int64


In [ ]:
print(train_kannada_data['category'].value_counts())

Non-anti-LGBT+ content    4463
Transphobia               2835
Homophobia                2765
Name: category, dtype: int64


In [ ]:
print(train_gujarati_data['category'].value_counts())

Non-anti-LGBT+ content    3848
Homophobia                2267
Transphobia               2004
Name: category, dtype: int64


In [ ]:
print(train_marathi_data['category'].value_counts())

Non-anti-LGBT+ content    2572
Homophobia                 551
Transphobia                377
Name: category, dtype: int64


In [ ]:
print(train_spanish_data['category'].value_counts())

Non-anti-LGBT+ content    700
Transphobia               250
Homophobia                250
Name: category, dtype: int64


In [ ]:
from google.cloud import translate_v2 as translate
import random

translate_client = translate.Client()

def back_translate(text, target_language, intermediate_language):
    translated = translate_client.translate(text, source_language=target_language, target_language=intermediate_language)
    translated_text = translated['translatedText']

    back_translated = translate_client.translate(translated_text, source_language=intermediate_language, target_language=target_language)
    back_translated_text = back_translated['translatedText']

    return back_translated_text


def get_homo_trans_new_df(dataframe, desired_categories):
    filtered_df = dataframe[dataframe['category'].isin(desired_categories)]
    return filtered_df


def translate_and_add_to_df(train_data, target_lg, intermed_lg, desired_categories):
    for index, row in get_homo_trans_new_df(train_data, desired_categories).iterrows():
        augmented_text = back_translate(row['text'], target_language=target_lg, intermediate_language=intermed_lg)
        df2 = {'text': augmented_text, 'category': row['category']}
        train_data = train_data.append(df2, ignore_index = True)
    return train_data

def translate_and_add_to_df_fixed_nr(train_data, target_lg, intermed_lg, desired_categories, limit):
    for index, row in get_homo_trans_new_df(train_data, desired_categories).iterrows():
        if limit == 0:
          return train_data
        augmented_text = back_translate(row['text'], target_language=target_lg, intermediate_language=intermed_lg)
        df2 = {'text': augmented_text, 'category': row['category']}
        train_data = train_data.append(df2, ignore_index = True)
        limit -= 1
    return train_data

In [ ]:
desired_categories = ['Homophobia', 'Transphobia']

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(train_eng_data, 'en', 'fr', desired_categories)

NameError: name 'train_augmented_train_eng_dataeng_data' is not defined

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'ro', desired_categories)

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'el', desired_categories)

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'ja', desired_categories)

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia                112
Name: category, dtype: int64


In [ ]:
augmented_train_tam_data = translate_and_add_to_df(train_tam_data, 'ta', 'en', desired_categories)
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                 906
Transphobia                290
Name: category, dtype: int64


In [ ]:
augmented_train_tam_data = translate_and_add_to_df(augmented_train_tam_data, 'ta', 'ja', desired_categories)
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                1812
Transphobia                580
Name: category, dtype: int64


In [ ]:
augmented_train_mal_data = translate_and_add_to_df(train_mal_data, 'ml', 'en', desired_categories)
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                1812
Transphobia                580
Name: category, dtype: int64


In [ ]:
augmented_train_mal_data = translate_and_add_to_df(augmented_train_mal_data, 'ml', 'ja', desired_categories)
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                1812
Transphobia                580
Name: category, dtype: int64


In [ ]:
augmented_train_hin_data = translate_and_add_to_df(train_hin_data, 'hi', 'en', desired_categories)
print(augmented_train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia                112
Name: category, dtype: int64


In [ ]:
augmented_train_hin_data = translate_and_add_to_df(augmented_train_hin_data, 'hi', 'ja', desired_categories)

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia                112
Name: category, dtype: int64


In [ ]:
augmented_train_hin_data = translate_and_add_to_df(augmented_train_hin_data, 'hi', 'ru', desired_categories)

In [ ]:
augmented_train_hin_data = translate_and_add_to_df(augmented_train_hin_data, 'hi', 'sr', desired_categories)

In [ ]:
print(augmented_train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2423
Transphobia               1472
Homophobia                 720
Name: category, dtype: int64


In [ ]:
augmented_train_marathi_data = translate_and_add_to_df(train_marathi_data, 'mr', 'en', desired_categories)

In [ ]:
augmented_train_marathi_data = translate_and_add_to_df(augmented_train_marathi_data, 'mr', 'ja', desired_categories)

In [ ]:
print(augmented_train_marathi_data['category'].value_counts())

Non-anti-LGBT+ content    2572
Homophobia                2204
Transphobia               1508
Name: category, dtype: int64


In [ ]:
augmented_train_spanish_data = translate_and_add_to_df(train_spanish_data, 'es', 'en', desired_categories)

In [ ]:
print(augmented_train_spanish_data['category'].value_counts())

Non-anti-LGBT+ content    700
Transphobia               500
Homophobia                500
Name: category, dtype: int64


In [ ]:
#save augmented ds
augmented_train_eng_data.to_csv('aug_translate_train_eng_data.csv', index=False)

In [ ]:
augmented_train_tam_data.to_csv('aug_translate_train_tam_data.csv', index=False)

In [ ]:
augmented_train_tam_data.to_csv('aug_translate_train_tam_data.csv', index=False)
augmented_train_mal_data.to_csv('aug_translate_train_mal_data.csv', index=False)
augmented_train_hin_data.to_csv('aug_translate_train_hin_data.csv', index=False)
augmented_train_marathi_data.to_csv('aug_translate_train_marathi_data.csv', index=False)
augmented_train_spanish_data.to_csv('aug_translate_train_spanish_data.csv', index=False)

In [ ]:
#new_train_data = translate_and_add_to_df(train_data, target_lg='en', intermed_lg='fr')
#print(new_train_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                 358
Transphobia                 14
Name: category, dtype: int64


In [ ]:
#new_dev_data = translate_and_add_to_df(dev_data, target_lg='en', intermed_lg='fr')
#print(new_dev_data['category'].value_counts())

NameError: name 'translate_and_add_to_df' is not defined

In [ ]:
# New day, new idea.. Further translate under represented categories aka Transphobic so that we get a bit more balanced before
# the actual oversampling process.


# get my saved aug csv's
augmented_train_eng_data = load_data('aug_translate_train_eng_data.csv')
augmented_train_hin_data = load_data('aug_translate_train_hin_data.csv')
augmented_train_mal_data = load_data('aug_translate_train_mal_data.csv')
augmented_train_marathi_data = load_data('aug_translate_train_marathi_data.csv')
augmented_train_spanish_data = load_data('aug_translate_train_spanish_data.csv')
augmented_train_tam_data = load_data('aug_translate_train_tam_data.csv')

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia                112
Name: category, dtype: int64


In [ ]:
desired_categories = ['Transphobia']
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'fi', desired_categories)

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'bg', desired_categories)

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia                448
Name: category, dtype: int64


In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'hr', desired_categories)

In [ ]:
augmented_train_eng_data = translate_and_add_to_df(augmented_train_eng_data, 'en', 'pl', desired_categories)

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2864
Transphobia               1792
Name: category, dtype: int64


In [ ]:
# now make it 100% balanced
augmented_train_eng_data = translate_and_add_to_df_fixed_nr(augmented_train_eng_data, 'en', 'ru', ['Transphobia'], 1186)

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Transphobia               2978
Homophobia                2864
Name: category, dtype: int64


In [ ]:
augmented_train_eng_data = translate_and_add_to_df_fixed_nr(augmented_train_eng_data, 'en', 'ru', ['Homophobia'], 114)

In [ ]:
print(augmented_train_eng_data['category'].value_counts())

Non-anti-LGBT+ content    2978
Homophobia                2978
Transphobia               2978
Name: category, dtype: int64


In [ ]:
# done with the english dataset

In [ ]:
print(augmented_train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2423
Transphobia               1472
Homophobia                 720
Name: category, dtype: int64


In [ ]:
augmented_train_hin_data = translate_and_add_to_df(augmented_train_hin_data, 'hi', 'fi', ['Homophobia'])

In [ ]:
print(augmented_train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2423
Transphobia               1472
Homophobia                1440
Name: category, dtype: int64


In [ ]:
#make it 100% balanced
augmented_train_hin_data = translate_and_add_to_df_fixed_nr(augmented_train_hin_data, 'hi', 'sv', ['Homophobia'], 983)

In [ ]:
augmented_train_hin_data = translate_and_add_to_df_fixed_nr(augmented_train_hin_data, 'hi', 'sv', ['Transphobia'], 951)

In [ ]:
print(augmented_train_hin_data['category'].value_counts())

Non-anti-LGBT+ content    2423
Homophobia                2423
Transphobia               2423
Name: category, dtype: int64


In [ ]:
# done with the HIN dataset

In [ ]:
print(augmented_train_mal_data['category'].value_counts())

Non-anti-LGBT+ content    2468
Homophobia                1904
Transphobia               1360
Name: category, dtype: int64


In [ ]:
augmented_train_mal_data = translate_and_add_to_df_fixed_nr(augmented_train_mal_data, 'ml', 'sv', ['Homophobia'], 564)

In [ ]:
augmented_train_mal_data = translate_and_add_to_df_fixed_nr(augmented_train_mal_data, 'ml', 'sv', ['Transphobia'], 1108)

In [ ]:
print(augmented_train_mal_data['category'].value_counts())

Non-anti-LGBT+ content    2468
Transphobia               2468
Homophobia                2468
Name: category, dtype: int64


In [ ]:
#done with mal dataset

In [ ]:
print(augmented_train_marathi_data['category'].value_counts())

Non-anti-LGBT+ content    2572
Homophobia                2204
Transphobia               1508
Name: category, dtype: int64


In [ ]:
augmented_train_marathi_data = translate_and_add_to_df_fixed_nr(augmented_train_marathi_data, 'mr', 'sv', ['Homophobia'], 368)

In [ ]:
augmented_train_marathi_data = translate_and_add_to_df_fixed_nr(augmented_train_marathi_data, 'mr', 'sv', ['Transphobia'], 1064)

In [ ]:
print(augmented_train_marathi_data['category'].value_counts())

Homophobia                2572
Non-anti-LGBT+ content    2572
Transphobia               2572
Name: category, dtype: int64


In [ ]:
#done with marathi dataset

In [ ]:
print(augmented_train_spanish_data['category'].value_counts())

Non-anti-LGBT+ content    700
Transphobia               500
Homophobia                500
Name: category, dtype: int64


In [ ]:
augmented_train_spanish_data = translate_and_add_to_df_fixed_nr(augmented_train_spanish_data, 'es', 'sv', ['Homophobia'], 200)

In [ ]:
augmented_train_spanish_data = translate_and_add_to_df_fixed_nr(augmented_train_spanish_data, 'es', 'sv', ['Transphobia'], 200)

In [ ]:
print(augmented_train_spanish_data['category'].value_counts())

Non-anti-LGBT+ content    700
Transphobia               700
Homophobia                700
Name: category, dtype: int64


In [ ]:
#done with spanish dataset

In [ ]:
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Homophobia                1812
Transphobia                580
Name: category, dtype: int64


In [ ]:
# first add more samples to transphobia --> we ll have 1160
augmented_train_tam_data = translate_and_add_to_df(augmented_train_tam_data, 'ta', 'ar', ['Transphobia'])

In [ ]:
augmented_train_tam_data = translate_and_add_to_df_fixed_nr(augmented_train_tam_data, 'ta', 'sv', ['Homophobia'], 252)

In [ ]:
augmented_train_tam_data = translate_and_add_to_df_fixed_nr(augmented_train_tam_data, 'ta', 'sv', ['Transphobia'], 904)

In [ ]:
print(augmented_train_tam_data['category'].value_counts())

Non-anti-LGBT+ content    2064
Transphobia               2064
Homophobia                2064
Name: category, dtype: int64


In [ ]:
#done with tam dataset

In [ ]:
# I did not augment these but I might
train_telugu_data = load_data('homo_Telugu_train.csv')
train_kannada_data = load_data('homo_Kannada_train.csv')
train_gujarati_data = load_data('homo_Gujarati_train.csv')

In [ ]:
print(train_telugu_data['category'].value_counts())

Non-anti-LGBT+ content    3496
Homophobia                2907
Transphobia               2647
Name: category, dtype: int64


In [ ]:
augmented_train_telugu_data = translate_and_add_to_df_fixed_nr(train_telugu_data, 'te', 'en', ['Homophobia'], 589)

In [ ]:
augmented_train_telugu_data = translate_and_add_to_df_fixed_nr(augmented_train_telugu_data, 'te', 'en', ['Transphobia'], 849)

In [ ]:
print(augmented_train_telugu_data['category'].value_counts())

Non-anti-LGBT+ content    3496
Transphobia               3496
Homophobia                3496
Name: category, dtype: int64


In [ ]:
#done with telugu dataset

In [ ]:
print(train_kannada_data['category'].value_counts())

Non-anti-LGBT+ content    4463
Transphobia               2835
Homophobia                2765
Name: category, dtype: int64


In [ ]:
augmented_train_kannada_data = translate_and_add_to_df_fixed_nr(train_kannada_data, 'kn', 'en', ['Homophobia'], 1698)

In [ ]:
augmented_train_kannada_data = translate_and_add_to_df_fixed_nr(augmented_train_kannada_data, 'kn', 'en', ['Transphobia'], 1628)

In [ ]:
print(augmented_train_kannada_data['category'].value_counts())

Homophobia                4463
Transphobia               4463
Non-anti-LGBT+ content    4463
Name: category, dtype: int64


In [ ]:
#done with kannada dataset

In [ ]:
print(train_gujarati_data['category'].value_counts())

Non-anti-LGBT+ content    3848
Homophobia                2267
Transphobia               2004
Name: category, dtype: int64


In [ ]:
augmented_train_gujarati_data = translate_and_add_to_df_fixed_nr(train_gujarati_data, 'gu', 'en', ['Homophobia'], 1581)

In [ ]:
augmented_train_gujarati_data = translate_and_add_to_df_fixed_nr(augmented_train_gujarati_data, 'gu', 'en', ['Transphobia'], 1844)

In [ ]:
print(augmented_train_gujarati_data['category'].value_counts())

Non-anti-LGBT+ content    3848
Homophobia                3848
Transphobia               3848
Name: category, dtype: int64


In [ ]:
#done with gujarati dataset

In [ ]:
# shuffle all of them?
balanced_augmented_train_eng_data = augmented_train_eng_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_tam_data = augmented_train_tam_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_mal_data = augmented_train_mal_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_hin_data = augmented_train_hin_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_marathi_data = augmented_train_marathi_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_spanish_data = augmented_train_spanish_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_telugu_data = augmented_train_telugu_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_kannada_data = augmented_train_kannada_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_gujarati_data = augmented_train_gujarati_data.sample(frac=1).reset_index(drop=True)

In [ ]:
#now we have 100% balance between classes in all languages. Next --> oversampling so that we have the same amount of comments in all languages
#maximum: with a total of samples for all 3 classes
dfs = [balanced_augmented_train_eng_data, balanced_augmented_train_tam_data, balanced_augmented_train_mal_data,
       balanced_augmented_train_hin_data, balanced_augmented_train_marathi_data, balanced_augmented_train_spanish_data,
       balanced_augmented_train_telugu_data, balanced_augmented_train_kannada_data, balanced_augmented_train_gujarati_data]

In [ ]:
balanced_augmented_train_eng_data.to_csv('balanced_augmented_train_eng_data.csv', index=False)
balanced_augmented_train_tam_data.to_csv('balanced_augmented_train_tam_data.csv', index=False)
balanced_augmented_train_mal_data.to_csv('balanced_augmented_train_mal_data.csv', index=False)
balanced_augmented_train_hin_data.to_csv('balanced_augmented_train_hin_data.csv', index=False)
balanced_augmented_train_marathi_data.to_csv('balanced_augmented_train_marathi_data.csv', index=False)
balanced_augmented_train_spanish_data.to_csv('balanced_augmented_train_spanish_data.csv', index=False)
balanced_augmented_train_telugu_data.to_csv('balanced_augmented_train_telugu_data.csv', index=False)
balanced_augmented_train_kannada_data.to_csv('balanced_augmented_train_kannada_data.csv', index=False)
balanced_augmented_train_gujarati_data.to_csv('balanced_augmented_train_gujarati_data.csv', index=False)

final_training_set = pd.concat(dfs, ignore_index=True)

In [ ]:

final_training_set = pd.concat(dfs, ignore_index=True)

In [ ]:
from sklearn.utils import resample
max_rows = 4463

oversampled_dfs = []
for df in dfs:
    oversampled_grouped = df.groupby('category', group_keys=False).apply(lambda x: resample(x, replace=True, n_samples=max_rows))
    oversampled_dfs.append(oversampled_grouped)

for i, oversampled_df in enumerate(oversampled_dfs):
    print(len(oversampled_df))

13389
13389
13389
13389
13389
13389
13389
13389
13389


In [ ]:
#concatenate everything and get one single dataset for training
final_training_set = pd.concat(oversampled_dfs, ignore_index=True)

In [ ]:
final_training_set = final_training_set.sample(frac=1).reset_index(drop=True)

In [ ]:
final_training_set.to_csv('final_training_set_without_OS_shuffled.csv', index=False)

In [ ]:
#oversample at category level, independently for each language set

# def oversample_to_third(df, target_column='category'):
#     # Calculate the new size for each class (count of Non-anti-LGBT+ content)
#     new_size_per_class = df['category'].value_counts()['Non-anti-LGBT+ content']
#     print(new_size_per_class)

#     # Oversample smaller classes to this new size
#     resampled_dfs = [resample(df[df[target_column] == cls],
#                               replace=True,
#                               n_samples=new_size_per_class,
#                               random_state=123)
#                      if len(df[df[target_column] == cls]) < new_size_per_class
#                      else df[df[target_column] == cls]
#                      for cls in ['Homophobia', 'Transphobia', 'Non-anti-LGBT+ content']]

#     # Combine resampled DataFrames
#     balanced_df = pd.concat(resampled_dfs, ignore_index=True)
#     return balanced_df

# # Balance the classes
# balanced_augmented_train_eng_data = oversample_to_third(augmented_train_eng_data)

# # Verify the new class distribution
# print(balanced_augmented_train_eng_data['category'].value_counts())

2978
Homophobia                2978
Transphobia               2978
Non-anti-LGBT+ content    2978
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_tam_data = oversample_to_third(augmented_train_tam_data)
# print(balanced_augmented_train_tam_data['category'].value_counts())

2064
Homophobia                2064
Transphobia               2064
Non-anti-LGBT+ content    2064
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_mal_data = oversample_to_third(augmented_train_mal_data)
# print(balanced_augmented_train_mal_data['category'].value_counts())

2468
Homophobia                2468
Transphobia               2468
Non-anti-LGBT+ content    2468
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_hin_data = oversample_to_third(augmented_train_hin_data)
# print(balanced_augmented_train_hin_data['category'].value_counts())

2423
Homophobia                2423
Transphobia               2423
Non-anti-LGBT+ content    2423
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_marathi_data = oversample_to_third(augmented_train_marathi_data)
# print(balanced_augmented_train_marathi_data['category'].value_counts())

2572
Homophobia                2572
Transphobia               2572
Non-anti-LGBT+ content    2572
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_spanish_data = oversample_to_third(augmented_train_spanish_data)
# print(balanced_augmented_train_spanish_data['category'].value_counts())

700
Homophobia                700
Transphobia               700
Non-anti-LGBT+ content    700
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_telugu_data = oversample_to_third(train_telugu_data)
# print(balanced_augmented_train_telugu_data['category'].value_counts())

3496
Homophobia                3496
Transphobia               3496
Non-anti-LGBT+ content    3496
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_kannada_data = oversample_to_third(train_kannada_data)
# print(balanced_augmented_train_kannada_data['category'].value_counts())

4463
Homophobia                4463
Transphobia               4463
Non-anti-LGBT+ content    4463
Name: category, dtype: int64


In [ ]:
# balanced_augmented_train_gujarati_data = oversample_to_third(train_gujarati_data)
# print(balanced_augmented_train_gujarati_data['category'].value_counts())

3848
Homophobia                3848
Transphobia               3848
Non-anti-LGBT+ content    3848
Name: category, dtype: int64


In [ ]:
balanced_augmented_train_eng_data = balanced_augmented_train_eng_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_tam_data = balanced_augmented_train_tam_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_mal_data = balanced_augmented_train_mal_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_hin_data = balanced_augmented_train_hin_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_marathi_data = balanced_augmented_train_marathi_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_spanish_data = balanced_augmented_train_spanish_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_telugu_data = balanced_augmented_train_telugu_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_kannada_data = balanced_augmented_train_kannada_data.sample(frac=1).reset_index(drop=True)
balanced_augmented_train_gujarati_data = balanced_augmented_train_gujarati_data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(balanced_augmented_train_eng_data)

                                                   text  \
0     Inta mutta pundayinga tola tanga mudira bus la...   
1                                        any btm sucker   
2     You tried to make a joke with a normal girl, b...   
3     Background music is so good I like this movie ...   
4     All the transferees say that they are prostitu...   
...                                                 ...   
8929         Wait is meera really trans                   
8930  Brow  I love ur act  Such a natural Act       ...   
8931  IN india how many of them folowing our culture...   
8932  Im crying more effective emotions                   
8933  Hii so called intellectual has nothing to say ...   

                    category  
0                 Homophobia  
1                 Homophobia  
2                Transphobia  
3     Non-anti-LGBT+ content  
4                Transphobia  
...                      ...  
8929  Non-anti-LGBT+ content  
8930  Non-anti-LGBT+ content  
8931  Non-an

In [ ]:
#concatenate everything and get one single dataset for training
pdList = [balanced_augmented_train_eng_data, balanced_augmented_train_tam_data,balanced_augmented_train_mal_data,
          balanced_augmented_train_hin_data, balanced_augmented_train_marathi_data, balanced_augmented_train_spanish_data,
          balanced_augmented_train_telugu_data, balanced_augmented_train_kannada_data, balanced_augmented_train_gujarati_data]
final_training_set = pd.concat(pdList, ignore_index=True)

In [ ]:
final_training_set = final_training_set.sample(frac=1).reset_index(drop=True)

In [ ]:
print(final_training_set)

                                                    text  \
0      சுற்றத்தை கெடுத்து gay ahh வாழ்வதை விட  அடுத்த...   
1           ટ્રાન્સજેન્ડરો સમાજના મૂલ્યોનો અનાદર કરે છે.   
2        તમે ટ્રાન્સજેન્ડર છો? તમને શાને કરવામાં આવે છે?   
3                పెళ్లిపై ఆమిర్‌కు సల్మాన్ గట్టి కౌంటర్!   
4      તેમની જાતિ કેવી રીતે તેમની જેન્ડર ઓળખને પ્રભાવ...   
...                                                  ...   
75031  Rajeswari Rajagopalan Sister Last week when I ...   
75032  તમે ટ્રાન્સજેન્ડર છો એટલે તમને અસલી માણસ તરીકે...   
75033  Venkat Raman போடா மெண்டல்  9 துங்ககிட்ட நீ அசி...   
75034  இது வரவேற்கத்தக்க தீர்ப்பு ஏனென்றால் தன்பாலின ...   
75035  રસ્તા પર અચાનક નાચવા લાગી ટીવી એકટ્રેસ, 6 લાખ ...   

                     category  
0                  Homophobia  
1                 Transphobia  
2                 Transphobia  
3      Non-anti-LGBT+ content  
4                 Transphobia  
...                       ...  
75031             Transphobia  
75032             Trans

In [ ]:
#final_training_set = load_data('final_training_set.csv')

In [ ]:
final_training_set.to_csv('final_training_set.csv', index=False)